<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/airport2.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 예측합니다.
- 성능이 좋을 것으로 판단된 모델의 성능을 튜닝합니다.
- 튜닝된 모델의 성능을 평가합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/airline_satisfaction_small.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**1) 상위 데이터 확인**

- 상위 일부 데이터를 확인합니다.

In [ ]:
# 데이터 살펴보기
data.head()

**데이터 설명**

- id : 탑승자 고유 아이디
- gender: 성별 (Female, Male)
- customer_type: 고객 유형 (Loyal customer, disloyal customer)
- age: 탑승자 나이
- type_of_travel: 비행 목적(Personal Travel, Business Travel)
- class: 등급 (Business, Eco, Eco Plus)
- flight_distance: 비행 거리
- inflight_wifi_service: 와이파이 서비스 만족도 (0:N/A; 1-5)
- departure/arrival_time_convenient: 출발, 도착 시간 만족도 (0:N/A; 1-5)
- ease_of_online_booking: 온라인 부킹 만족도 (0:N/A; 1-5)
- gate_location: 게이트 위치 만족도 (0:N/A; 1-5)
- food_and_drink: 식사와 음료 만족도 (0:N/A; 1-5)
- online_boarding: 온라인 보딩 만족도 (0:N/A; 1-5)
- seat_comfort: 좌석 편안함 만족도 (0:N/A; 1-5)
- inflight_entertainment: 기내 엔터테인먼트 만족도 (0:N/A; 1-5)
- on-board_service: 온 보드 서비스 만족도 (0:N/A; 1-5)
- leg_room_service: 다리 공간 만족도 (0:N/A; 1-5)
- baggage_handling: 수하물 처리 만족도 (0:N/A; 1-5)
- check-in_service: 체크인 서비스 만족도 (0:N/A; 1-5)
- inflight_service: 기내 서비스 만족도 (0:N/A; 1-5)
- cleanliness: 청결 만족도 (0:N/A; 1-5)
- departure_delay_in_minutes: 출발 지연 시간(분)
- arrival_delay_in_minutes: 도착 지연 시간(분)
- satisfaction: 항공 만족도(1: Satisfaction, 0: Neutral or Dissatisfaction) - Target

**2) 열 정보 확인**

- 모든 열의 이름, 데이터 형식, 데이터 개수 등을 확인합니다.

In [ ]:
# 열 정보 확인
data.info()

**3) Target 변수 분포 확인**

- Target 변수의 값 분포를 시각화해 확인합니다.

In [ ]:
# Target 변수 값 분포 확인
sns.countplot(x=data['satisfaction'])
plt.show()

**4) 문자열 범줏값 확인**

- 문자열 범줏값을 갖는 변수들의 값 분포를 시각화해 확인합니다.

In [ ]:
# 문자열 변수
dtypes = data.dtypes
cols = dtypes.loc[dtypes=='object'].index

# 시각화
rows = (len(cols) + 1) // 2
plt.figure(figsize=(8, 2* rows))
for idx, col in enumerate(cols):
    plt.subplot(rows, 2, idx+1)
    sns.countplot(x=data[col])
plt.tight_layout()
plt.show()

**5) 숫자 범줏값 확인**

- 숫자 범줏값을 갖는 설문조사 관련 변수들의 응답 분포를 시각화해 확인합니다.

In [ ]:
# 설문조사 변수
dtypes = data.dtypes
cols = data.loc[:, 'inflight_wifi_service': 'cleanliness'].columns

# 시각화
rows = (len(cols) + 1) // 2
plt.figure(figsize=(8, 2 * rows))
for idx, col in enumerate(cols):
    plt.subplot(rows, 2, idx+1)
    sns.countplot(x=data[col])
plt.tight_layout()
plt.show()

**6) 기술 통계 확인**

- 기술 통계를 사용해 수치형 변수들의 값 분포를 확인합니다.
- 위에서 살펴본 범줏값을 갖는변수를 제외하고 확인합니다.

In [ ]:
# 기술통계 확인
cols = ['age', 'flight_distance','departure_delay_in_minutes', 'arrival_delay_in_minutes']
data[cols].describe().T

**7) 수치형 변수 값 분포 확인**

- 위 수치형 변수들의 값 분포와 Target 변수와의 관계를 시각화해 확인합니다.

In [ ]:
# 수치형 변수
cols = ['age', 'flight_distance','departure_delay_in_minutes', 'arrival_delay_in_minutes']

# 시각화
rows = (len(cols) + 1) // 2
plt.figure(figsize=(8, 2 * rows))
for idx, col in enumerate(cols):
    plt.subplot(rows, 2, idx+1)
    sns.histplot(x=col, hue='satisfaction', bins=30, data=data)
plt.tight_layout()
plt.show()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

- 다음 변수들은 만족도 예측에 미지는 영향이 적다고 판단되므로 제거합니다.
    - id, departure/arrival_time_convenient, gate_location, arrival_delay_in_minutes, departure_delay_in_minutes

In [ ]:
# 제거 대상 변수
del_cols = ['id', 'departure/arrival_time_convenient', 'gate_location', 'arrival_delay_in_minutes', 'departure_delay_in_minutes']

# 변수 제거
data.drop(del_cols, axis=1, inplace=True)

# 확인
data.head()

**2) 결측치 확인**

- 결측치를 가졌던 arrival_delay_in_minutes 변수가 위 과정에서 제거된 상태이므로 결측치가 없습니다.
- 결측치 존재 여부를 최종 확인하고 진행합니다.

In [ ]:
# 결측치 확인
data.isnull().sum()

**3) x, y 분리**

- 전체 데이터를 x, y로 분리하고, 이후 과정에서는 x, y만 사용합니다.

In [ ]:
# Target 설정
target = 'satisfaction'

# 데이터 분리
x = data.drop(target, axis=1)
y = data.loc[:, target]

**4) 가변수화**

- 다음 문자열 변수를 가변수화 합니다.
    - gender, customer_type, type_of_travel, class

In [ ]:
# 가변수화 대상 변수
dumm_cols =['gender', 'customer_type', 'type_of_travel', 'class']

# 가변수화
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**5) 학습용, 평가용 데이터 분리**

- 학습용, 평가용 데이터를 7:3으로 분리합니다.

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

**6) 정규화**

- KNN 모델을 위해 정규화 된 결과를 별도로 준비합니다.

In [ ]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)

# 4.성능 예측

- 여러 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 예측합니다.
- 하이퍼파라미터를 설정하지 않고 진행합니다.
- 각 모델의 성능 정보를 수집해 마지막에 비교합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import *

**1) KNN**

- KNN 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = KNeighborsClassifier()

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train_s, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result = {}
result['KNN'] = cv_score.mean()

**2) Decision Tree**

- Decision Tree 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = DecisionTreeClassifier()

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Decision Tree'] = cv_score.mean()

**3) Logistic Regression**

- Logistic Regression 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = LogisticRegression()

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Logistic Regression'] = cv_score.mean()

**3) Random Forest**

- Random Forest 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = RandomForestClassifier()

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Random Forest'] = cv_score.mean()

**4) XGBoost**

- XGBoost 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = XGBClassifier()

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['XGBoost'] = cv_score.mean()

**5) LightGBM**

- LightGBM 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [ ]:
# 선언하기
model = LGBMClassifier(verbose=-1)

In [ ]:
# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['LightGBM'] = cv_score.mean()

# 5.결과 확인

- 예측된 각 모델의 성능을 비교합니다.

In [ ]:
# 성능 비교
print('=' * 40)
for m_name, score in result.items():
    print(m_name, score.round(3))
print('=' * 40)

# 6.성능 튜닝

- 위에서 성능이 가장 좋을 것으로 예측된 모델을 튜닝합니다.
- 본 실습에서는 XGBoost 모델 성능을 튜닝합니다.

In [ ]:
# 파라미터 지정
  # max_depth: range(1, 21)
param = {'max_depth': range(1, 21)}

# 모델 선언
model = GridSearchCV(XGBClassifier(),
                     param,
                     cv=5,
                     scoring='accuracy')

In [ ]:
# 학습하기(많은 시간이 소요될 수 있음)
model.fit(x_train, y_train)

In [ ]:
# 최적 파라미터, 예측 최고 성능
print(model.best_params_)
print(model.best_score_)

In [ ]:
# 변수 중요도 시각화
df = pd.DataFrame()
df['feature'] = list(x)
df['importance'] =  model.best_estimator_.feature_importances_
df.sort_values(by='importance', ascending=True, inplace=True)
plt.figure(figsize=(5, 5))
plt.barh(y=df['feature'], width=df['importance'])
plt.show()

# 7.성능 평가

- 최적 파라미터로 학습된 모델에 대해 최종 성능 평가를 진행합니다.

In [ ]:
# 예측하기
y_pred = model.predict(x_test)

In [ ]:
# 성능평가
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))